In [12]:
#export
#General imports
import pandas as pd
import numpy as np
import os
import pickle
import sys
import exporter
from datetime import datetime, timedelta

sys.path.append('../src/')
import project_constants 


#email sending
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

#logging
import logging
# Gets or creates a logger
logger = logging.getLogger(__name__)  
# set log level
logger.setLevel(logging.INFO)
# define file handler and set formatter
file_handler = logging.FileHandler('logfile.log')
formatter    = logging.Formatter('%(asctime)s : %(levelname)s : %(name)s : %(message)s')
file_handler.setFormatter(formatter)
# add file handler to logger
logger.addHandler(file_handler)

#StringMatch
from fuzzywuzzy import fuzz

#Google Trends & Tweeter  
from pytrends.request import TrendReq
import tweepy

#environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# export
def f_send_mail(mail_content=None, 
                mail_to=None,
                mail_subject=None):
    if mail_to is None:
        mail_to = 'xxxxxxxxx@gmail.com'
        
    if mail_content is None:
        mail_content = 'Just try mail sending logic. No specific content is set yet.'    
        
    if mail_subject is None:
        mail_subject = 'Test Mail from App'    

    logger.info('Try to send email to {x}'.format(x=mail_to))
    #mail_content = '''Hello,
    #This is a simple mail. There is only text, no attachments are there The mail is sent using Python SMTP library.
    #Thank You
    #'''
    
    #The mail addresses and password
    sender_address = os.environ.get('GMAIL_FROM_ADDRESS')
    sender_pass = os.environ.get('GMAIL_API_SECRET')
    receiver_address = mail_to
    
    message = MIMEMultipart()
    message['From'] = sender_address
    message['To'] = receiver_address
    message['Subject'] = mail_subject   #The subject line
    #The body and the attachments for the mail
    message.add_header('Content-Type','text/html')
    
    
    message.attach(MIMEText(mail_content, 'html'))
    try:
        #Create SMTP session for sending the mail
        session = smtplib.SMTP('smtp.gmail.com', 587) #use gmail with port
        session.starttls() #enable security
        session.login(sender_address, sender_pass) #login with mail_id and password
        text = message.as_string()
        session.sendmail(sender_address, receiver_address.split(";"), text)
        session.quit()
        logger.info('Mail Sent')
    except:
        logger.error('Sent mail has failed !!')
        session.quit()

In [19]:
#export
def connect_tweet():
    '''
    returns tweepy api
    '''
    # OAuth process, using the keys and tokens
    consumer_key = os.environ.get('TWEET_CONSUMER_KEY')
    consumer_secret = os.environ.get('TWEET_CONSUMER_SECRET')
    access_token = os.environ.get('TWEET_ACCESS_TOKEN')
    access_token_secret = os.environ.get('TWEET_ACCESS_TOKEN_SECRET')
    logger.info('Try to connect tweeter api')

    try:        
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        api = tweepy.API(auth)
        logger.info('Successfully connected to tweeter api')
    except:
        logger.error('tweeter connection has failed !!')
        print('ERROR!!!')
        
    return api



def get_tweet_trends_by_loc(v_api, v_loc):
    '''
    returns trend topics wrt given location in list
    '''
    if v_loc == "TR":
        woeid = 23424969
    elif v_loc == "US":
        woeid = 23424977    

    # fetching the trends
    trends = v_api.trends_place(id = woeid)
    list_tmp = []
    for value in trends:
        for trend in value['trends']:
            list_tmp.append(trend['name'])  
            #print(trend['name'])

    return list_tmp


def get_tweets_from_user(v_api, v_userid):
    '''
    returns last 24hour tweets from any specific twitter user in list
    '''
    # fetching the tweets
    list_tmp = []
    tweets = v_api.user_timeline(screen_name=v_userid, 
                               # 100 is the maximum allowed count
                               count=100,
                               include_rts = False,
                               # Necessary to keep full_text 
                               # otherwise only the first 140 words are extracted
                               #tweet_mode = 'extended'
                               )


    # printing the statuses
    for info in tweets:
        dt_hours_from_now = datetime.now() + timedelta(hours=-24)
        if info.created_at >= dt_hours_from_now: 
            txt = "ID: {id}, Date: {dt}, Text: {txt}".format(id=info.id, dt= info.created_at, txt= info.text )
            list_tmp.append(txt)
            #print("ID: {}".format(info.id))
            #print(info.created_at)
            #print(info.text, end = "\n\n")

    return list_tmp


def get_google_trends(pn='united_states'):
    '''
    returns current trend searches in given location in list
    '''
    pytrends = TrendReq()
    try:
        listx = list(pytrends.trending_searches(pn=pn).iloc[:, 0])
        logger.info('Successfully connected to GOOGLETRENDS API')
    except:
        listx = []
        logger.error('Failed to connect GOOGLETRENDS API')
    return listx 


def web_analyze_all():
    '''
    returns a dict including all google trends, twitter trends and specific user's tweets all together.
    '''
    api = connect_tweet()

    tweet_dict = {}
    tweet_dict.update({"TR TREND TOPICS": get_tweet_trends_by_loc(api, "TR")}) 
    tweet_dict.update({"US TREND TOPICS": get_tweet_trends_by_loc(api, "US")}) 
    
    #tweetuserid = 'elonmusk'
    for tweetuserid in project_constants.tweeter_user_list:
        tweet_dict.update({"TWEET from " + tweetuserid: get_tweets_from_user(api, tweetuserid)})
    
    tweet_dict.update({"Google US Trends ": get_google_trends()})
    
    return tweet_dict


def load_crypto_df():
    '''
    returns dataframe including all cryptocurrency symbol lookup together with some manually defined keywords
    '''
    coin_list_df = pd.read_csv("../lookup/crypto_lu.csv")
    coin_list_df["Symbolusd"] =  coin_list_df["Symbol"].apply(lambda x: x + "USDT" )
    coin_list_df["Keywords"]  = coin_list_df.apply(lambda row: row["Name"].lower() + "," + 
                               #row["Symbol"].lower() + "," + 
                               row["Symbolusd"].lower() + "," + 
                               row["Keywords"].lower()  , axis = 1)
    return coin_list_df
        
        

def match_symbol_with_tweets(v_score=90):

    coin_list_df = load_crypto_df()

    dict_tw = web_analyze_all()
    list_tmp=[]
    for index, row in coin_list_df.iterrows():
        for key in dict_tw:
            for trendtopic in dict_tw.get(key):
                list_query = row["Keywords"].split(",")
                #print(list_query)
                #print(trendtopic.lower())
                score = f_string_match(list_query, trendtopic.lower())
                #print(score)
                if score > v_score:
                    list_tmp.append([key, row["Name"], row["Keywords"], trendtopic.lower(), score])
                    #print([key, row["Keywords"], trendtopic.lower(), score])
                    #print("************************")
                #name_score = fuzz.token_set_ratio(row["Name"].lower(), trendtopic.lower())
                #symbol_score = fuzz.token_set_ratio(row["Symbol"].lower(), trendtopic.lower())
                #if name_score > v_score | symbol_score > v_score :
                #    list_tmp.append([key, row["Name"], row["Symbol"], trendtopic.lower(), name_score, symbol_score  ])
    
    return pd.DataFrame(data = list_tmp, columns =["Source","Coin", "Keywords", "News", "Score"]).sort_values("Coin")



def send_mails_with_matches(v_score_match = None, 
                            v_mail_subject = None,
                            v_to_address = None):
    if v_score_match is None:
        v_score = 90
    else:
        v_score = v_score_match
    df = match_symbol_with_tweets(v_score)
    #return df.to_html() 
    
    #for topic in v_list:
    #    mail_body = mail_body + ' $ '.join(topic[0:4]) + '\n'  
    if len(df) > 0:
        mail_body = df.to_html()
        f_send_mail(mail_content = mail_body, mail_subject = v_mail_subject, mail_to = v_to_address)
        print("successfully send mail")
    else:
        logger.info('Does not match anything, so we have nothing to sent!')
        print("Does not match anything, so we have nothing to sent!")


def f_string_match(list_query = ["dogecoin", "dogefatherx", "doge coin"], 
                   ref_string = "elonmust said dogefather!!"):
    max_score = 0
    new_max_score = 0
    for x in list_query:
        if len(x.split()) == 1:
            for ref_x in ref_string.split():
                if ref_x.startswith(x):
                    new_max_score = fuzz.ratio(x, ref_x) - 5  #penalty point
                #print("tek kelime {x} ve {ref_x} score: {score}".format(x=x, ref_x = ref_x, score = new_max_score))
                if new_max_score > max_score:
                    max_score = new_max_score
        else:    
            new_max_score = max( \
                               [fuzz.partial_ratio(x, ref_string),  
                               fuzz.token_set_ratio(x, ref_string), 
                               fuzz.token_sort_ratio(x, ref_string)
                               ]
                             )
            if new_max_score > max_score:
                max_score = new_max_score

    return max_score        
    

In [20]:
send_mails_with_matches(v_score_match = 75, 
                            v_mail_subject = "****sil*****",
                            v_to_address = 'xxxx@gmail.com')

successfully send mail


In [8]:
#f_string_match(['dolar kuru', 'usdtry', 'usdtryusdt', 'dolar'], "#dolar")

#match_symbol_with_tweets(v_score=90)

send_mails_with_matches(v_score_match = 75,
                        v_mail_subject = "WEB Trend Analysis Mail - SmartCryptoAnalysis",
                        v_to_address = "ali.alici84@gmail.com" )

#web_analyze_all()

#match_symbol_with_tweets(v_score=90)



#match_symbol_with_tweets(v_score=30)

#match_symbol_with_tweets(v_score=90)

successfully send mail


In [51]:
# export
if __name__ == "__main__":
    v_list = match_symbol_with_tweets(v_score=90)
    mail_body = ''
    for topic in v_list:
        mail_body = mail_body + ' $ '.join(topic[0:4]) + '\n'  
    if len(v_list) > 0:
        f_send_mail(mail_content = mail_body)

In [21]:
from exporter import export
export("GeneralFunctions.ipynb","../src/GeneralFunctions.py")